In [1]:
import pandas as pd
data = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)
print(data.shape)

In [3]:
data[:1]

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."


In [6]:
print(data.review[0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [14]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [23]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))

def review_to_words(review_raw):
    # 1. Remove HTML
    review_text = BeautifulSoup(review_raw).get_text()
    
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    # stops = set(stopwords.words("english"))
    
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.This is to make the output easier to 
    # use in our Bag of Words, below.
    return(" ".join(meaningful_words))

In [28]:
clean_train_reviews = []
num_reviews = len(data.review)
for i in xrange( num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews )                                                                    
    clean_train_reviews.append( review_to_words( data.review[i] ) )

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', max_features=5000)

data_features = vectorizer.fit_transform(clean_train_reviews)
data_features = data_features.toarray()

In [38]:
print(data_features.shape)

(25000L, 5000L)


In [43]:
vocab = vectorizer.get_feature_names()
print(vocab[:5])

[u'abandoned', u'abc', u'abilities', u'ability', u'able']


In [60]:
print "Training the random forest..."
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 100, n_jobs=-1, verbose=1, oob_score=True) 

clf_rf = rf.fit(data_features, data.sentiment )

Training the random forest...


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   38.1s finished


In [65]:
print('oob_score: %.2f%%' % (clf_rf.oob_score_*100))

oob_score: 82.61%


In [54]:
# read the test data
data_test = pd.read_csv('testData.tsv', sep='\t', quoting=3)

# get the clean_test_reviews
num_reviews_test = len(data_test)
clean_test_reviews = []
for i in xrange( num_reviews_test ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_reviews_test )                                                                    
    clean_test_reviews.append( review_to_words( data_test.review[i] ) )
    
# get a bag of words for the test set, and convert to a numpy array
data_test_features = vectorizer.transform(clean_test_reviews)
data_test_features = data_test_features.toarray()

#use the clf_rf to make sentiment label predictions
predictions = clf_rf.predict(data_test_features)

#create the ouput df, and write to the csv 
output = pd.DataFrame({'id': data_test.id, 'sentiment': predictions})
output.to_csv('Bag_of_Words_model_predictions.csv', index=False, quoting=3)

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000

Review 6000 of 25000

Review 7000 of 25000

Review 8000 of 25000

Review 9000 of 25000

Review 10000 of 25000

Review 11000 of 25000

Review 12000 of 25000

Review 13000 of 25000

Review 14000 of 25000

Review 15000 of 25000

Review 16000 of 25000

Review 17000 of 25000

Review 18000 of 25000

Review 19000 of 25000

Review 20000 of 25000

Review 21000 of 25000

Review 22000 of 25000

Review 23000 of 25000

Review 24000 of 25000

Review 25000 of 25000



[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.1s finished
